In [2]:
# pip install pymupdf4llm sentence-transformers chromadb ollama langchain
import pymupdf4llm
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PDF 파일 경로
file_path = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"
# 로컬 인공지능 언어 모델 
model = "exaone3.5:latest"

def load_pdf(file_path):
    pdf_data = pymupdf4llm.to_markdown(file_path)
    text = "".join(pdf_data)  # 페이지별 텍스트를 하나의 문자열로 결합
    print("PDF파일 로드..")
    return text

def split_text(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
    return text_splitter.split_text(text)

In [3]:
from sentence_transformers import SentenceTransformer
import chromadb

embedder = SentenceTransformer("intfloat/multilingual-e5-small")
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="rag_collection",  metadata={"hnsw:space": "cosine"} )

print("문서 로드 및 임베딩 시작...")

raw_text = load_pdf(file_path)
chunks = split_text(raw_text)
embeddings = embedder.encode(chunks, convert_to_tensor=False)

for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    collection.add(
        ids=[f"chunk_{i + 1}"],
        embeddings=[embedding.tolist()],
        metadatas=[{"text": chunk}],
    )

/Users/horus/dev/jupyter-root/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


문서 로드 및 임베딩 시작...
PDF파일 로드..


Add of existing embedding ID: chunk_1
Add of existing embedding ID: chunk_2
Add of existing embedding ID: chunk_3
Add of existing embedding ID: chunk_4
Add of existing embedding ID: chunk_5
Add of existing embedding ID: chunk_6
Add of existing embedding ID: chunk_7
Add of existing embedding ID: chunk_8
Add of existing embedding ID: chunk_9
Add of existing embedding ID: chunk_10
Add of existing embedding ID: chunk_11
Add of existing embedding ID: chunk_12
Add of existing embedding ID: chunk_13
Add of existing embedding ID: chunk_14
Add of existing embedding ID: chunk_15
Add of existing embedding ID: chunk_16
Add of existing embedding ID: chunk_17
Add of existing embedding ID: chunk_18
Add of existing embedding ID: chunk_19
Add of existing embedding ID: chunk_20
Add of existing embedding ID: chunk_21
Add of existing embedding ID: chunk_22
Add of existing embedding ID: chunk_1
Add of existing embedding ID: chunk_2
Add of existing embedding ID: chunk_3
Add of existing embedding ID: chunk_4

In [4]:
from collections import deque

class Message_manager:
    def __init__(self):
        self._system_msg = {"role": "system", "content": ""}
        self.queue = deque(maxlen=10)  # 최대 10개 대화 저장

    def create_msg(self, role, content):
        return {"role": role, "content": content}

    def system_msg(self, content):
        self._system_msg = self.create_msg("system", content)

    def append_msg(self, content):
        msg = self.create_msg("user", content)
        self.queue.append(msg)

    def get_chat(self):
        return [self._system_msg] + list(self.queue)

    def set_retrived_docs(self, docs):
        self.retrieved_docs = docs

    def append_msg_by_assistant(self, content):
        msg = self.create_msg("assistant", content)
        self.queue.append(msg)

    def generate_prompt(self, retrieved_docs):

        docs = "\n".join(retrieved_docs)

        prompt = [msgManager._system_msg,{
            "role": "system",
            "content": f"문서 내용: {docs}\n질문에 대한 답변은 문서 내용을 기반으로 정확히 제공하시오.",
        }] + list(msgManager.queue)

        return prompt


msgManager = Message_manager() # 객체 생성

msgManager.system_msg(
    "가장 마지막 'user'의 'content'에 대해 답변한다."
    "질문에 답할 때는 'system' 메시지 중 '문서 내용'에 명시된 부분을 우선 참고하여 정확히 답한다."
    "개행은 문장이 끝날때와 서로 다른 주제나 항목을 구분할 때 사용하며, 불필요한 개행은 넣지 않는다."
)

In [5]:
from langchain_community.chat_models import ChatOllama

# ollama 인스턴스 생성
ollama = ChatOllama(model=model)

def retrieve_docs(query, collection, embedder, top_k=2):
    query_embedding = embedder.encode(query, convert_to_tensor=False).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    
    if not results["metadatas"]:  # 검색 결과가 없는 경우
        return ["관련 문서를 찾을 수 없습니다."]
    
    docs = [doc["text"] for doc in results["metadatas"][0]]
    return docs

def generate_answer(query, retrieved_docs, conversation_history):
    # 이전 대화 기록 추가 (최대 10개)
    msgManager.append_msg(query)
    
    # 스트리밍으로 답변 생성
    print("답변: ", end="", flush=True)
    full_answer = ""
    
    msg = msgManager.generate_prompt(retrieved_docs)

    for response in ollama.stream(msg):
        chunk = getattr(response, "content", "")  # 안전하게 추출
        print(chunk, end="", flush=True)
        full_answer += chunk

    return full_answer

In [6]:
query = "대상 문서를 요약해주세요"

retrieved_docs = retrieve_docs(query, collection, embedder, top_k=2)

answer = generate_answer(query, retrieved_docs, msgManager.get_chat())
msgManager.append_msg_by_assistant(answer)

답변: **2023년 12월호 요약:**

**정책/법제:**
- 글로벌 차원에서 안전하고 신뢰할 수 있는 AI 개발과 사용을 위한 다양한 정책과 협정들이 발표되었다. 예를 들어, 미국은 AI 관련 행정명령을 발표했고, G7은 국제 행동강령에 합의했다. 유럽 연합(EU)의 경우, AI 법의 협상 과정에서 기반 모델 규제에 대한 의견 차이로 어려움을 겪고 있다.

**기업/산업 동향:**
- 여러 기업들이 AI 기술 발전에 힘쓰고 있다. 미국 프런티어 모델 포럼은 AI 안전 기금을 조성했으며, 알리바바 클라우드는 새로운 LLM 모델을 공개했다. 삼성전자는 자체 생성 AI 모델을 선보였고, 구글은 앤스로픽에 대규모 투자를 통해 협력을 강화했다. 시장 조사 기관 IDC는 2027년까지 AI 소프트웨어 매출이 2,500억 달러를 넘어설 것으로 전망했다. 또한, 유튜브는 2024년부터 AI 생성 콘텐츠에 대한 표시 의무화를 발표했다.

**기술/연구:**
- 구글 딥마인드는 범용 인공지능(AGI)의 성능과 범용성, 자율성을 기준으로 0~5단계까지 분류하는 프레임워크를 공개했다. 현재 AGI는 특정 분야에서는 고도의 성과를 보이지만, 폭넓은 범용성 측면에서는 초기 단계에 머물러 있다. 딥마인드는 AGI 발전의 공통 기준을 마련하기 위해 AI의 기능과 잠재력에 중점을 둔 6가지 원칙을 제시했다.

이러한 동향들은 AI 산업의 정책적, 기업적, 기술적 발전을 보여주며, 특히 AGI의 진전과 관련된 연구가 주목받고 있다.

In [ ]:
# from collections import deque

# def chat_loop():
#     print("RAG 챗봇 시작! 질문 입력 (종료하려면 'exit' 입력):")
    
#     while True:
#         query = input("> ")
#         if query.lower() == "exit":
#             print("챗봇 종료!")
#             break

#         retrieved_docs = retrieve_docs(query, collection, embedder, top_k=2)
        
#         answer = generate_answer(query, retrieved_docs, msgManager.get_chat())
#         msgManager.append_msg_by_assistant(answer)
        
# # 실행
# # if __name__ == "__main__":
# #     chat_loop()

# chat_loop()  # 챗봇 시작

RAG 챗봇 시작! 질문 입력 (종료하려면 'exit' 입력):


ValueError: Expected each embedding in the embeddings to be a list, got [array([ 7.52922818e-02, -3.29757780e-02, -9.79989097e-02, -1.26158595e-01,
        9.37714726e-02, -1.58334076e-02,  1.87153406e-02,  7.70311877e-02,
        5.97499423e-02,  1.84853803e-02,  3.72900292e-02,  3.72451060e-02,
        8.33959579e-02,  9.85871442e-03, -4.85196225e-02,  2.63275839e-02,
        5.59704788e-02, -3.71749513e-02, -3.25423814e-02, -2.97307037e-02,
       -3.27006564e-03,  7.95851927e-03, -4.67882119e-02,  6.90558106e-02,
        4.23402265e-02,  3.45860496e-02, -3.82249914e-02,  2.45692693e-02,
        4.13424000e-02, -5.78125417e-02, -5.00612855e-02, -4.46649529e-02,
        1.70833841e-02, -6.84955642e-02,  6.36692569e-02,  6.80206567e-02,
       -1.39026213e-02, -3.83295044e-02,  6.24954253e-02, -3.74409296e-02,
        5.07086422e-03,  8.71465076e-03,  6.38774410e-02,  6.97102919e-02,
        1.66848488e-02,  6.27657920e-02, -7.15160593e-02,  1.66916382e-02,
       -4.38218713e-02, -3.